In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('survey_results_public.csv')

### Скільки респондентів пройшли опитування?

In [ ]:
# варіант 1 - використовуючи shape
print("Total respondents:", df.shape[0])

# варіант 2 - використовуючи функцію unique()
# nunique() returns the number of unique values in a column
print("Total respondents:", df.ResponseId.nunique())

### Скільки респондентів відповіли на всі запитання?

!!! читаємо readme і дізнаємось які колонки відповідають питанням? - описані в survey_results_schema.csv

In [ ]:
schema = pd.read_csv("survey_results_schema.csv")
questions = set(schema.qname.unique()) & set(df.columns)  # вибираємо за допомогою множин тільки ті стовпці, які є в обох датафреймах

In [ ]:
schema = pd.read_csv("survey_results_schema.csv")
questions = set(schema.qname.unique()) & set(
    df.columns
)  # вибираємо за допомогою множин тільки ті стовпці, які є в обох датафреймах

# спосіб 1 - використовуючи підсемпл даних
print(f"Total respondents with full answers: {df[list(questions)].dropna().shape[0]}")

# спосіб 2 - використовуючи функцію dropna() видаляємо рядки, в яких є хоча б один пропуск, і рахуємо кількість рядків
print(f"Total respondents with full answers: {df.dropna(subset=questions).shape[0]}")

### Які значення мір центральної тенденції для досвіду (WorkExp) респондентів?

In [ ]:
print(f'Mean: {round(df.WorkExp.mean(), 1)}')
print(f'Mode: {round(df.WorkExp.mode()[0], 1)}')
print(f'Median: {round(df.WorkExp.median(), 1)}')

### Скільки респондентів працює віддалено?

In [ ]:
df["RemoteWork"].unique()

In [ ]:
df[df.RemoteWork == "Remote"].shape[0]

### Який відсоток респондентів програмує на Python?

In [ ]:
# Варіант 1 - використовуючи функцію .str.contains
df["worked_with_python"] = df.LanguageHaveWorkedWith.str.contains("python", case=False, na=False)  # case=False - не враховуємо регістр, na=False - не враховуємо пропуски
#round(df.worked_with_python.sum() / df.ResponseId.nunique(), 2)
round(df.worked_with_python.sum() / df.shape[0])

In [ ]:
# Варіант 1 - використовуючи функцію .str.contains попередньо привівши до нижнього регістру
df["LanguageHaveWorkedWith"] = df["LanguageHaveWorkedWith"].str.lower()
df["worked_with_python"] = df["LanguageHaveWorkedWith"].str.contains("python")


print(f"Total = {round(df[df['worked_with_python'] == True].shape[0] / df.shape[0], 4)}")
print(f"Total = {round(df.worked_with_python.sum() / df.shape[0], 4)}")

In [ ]:
df[["LanguageHaveWorkedWith", "worked_with_python"]]

In [ ]:
# Варіант 1.2 - використовуючи функцію .str.contains з параметрами

df["worked_with_python"] = df.LanguageHaveWorkedWith.str.contains(
    "python", case=False, na=False
)  # case=False - не враховуємо регістр, na=False - не враховуємо пропуски (домовленність розрахунків)

print(f"Total = {round(df.worked_with_python.sum() / df.shape[0], 4)}")
print(f"Total = {round(df.worked_with_python.sum() / df.ResponseId.nunique(), 4)}")

In [ ]:
# Варіант 2 - Використовуючи лямбда-функції для розмітки рядків
df["worked_with_python"] = df.LanguageHaveWorkedWith.apply(
    lambda x: 1 if "python" in str(x).lower() else 0
)

print(f"Total = {round(df.worked_with_python.sum() / df.shape[0], 4)}")
print(f"Total = {round(df.worked_with_python.sum() / df.ResponseId.nunique(), 4)}")

### Скільки респондентів навчалося програмувати за допомогою онлайн курсів?

In [ ]:
# Варіант 1 - використовуючи функцію .str.contains

print( f"Total={df[df['LearnCode'].str.contains('online courses', case=False, na=False)].shape[0]}")

In [ ]:
# Варіант 2

df["learned_with_online_courses"] = df.LearnCode.str.contains("online courses", case=False, na=False)

print(f"Total={df.learned_with_online_courses.sum()}")

In [ ]:
# Варіант 3 - використовуючи лямбда-функції
df["learned_with_online_courses"] = df.LearnCode.apply(lambda x: 1 if "online courses" in str(x).lower() else 0)

print(f"Total={df.learned_with_online_courses.sum()}")

### Серед респондентів що програмують на Python в групуванні по країнам, яка середня та медіанна сума компенсації (ConvertedCompYearly) в кожній країні?

In [ ]:
# варіант 1
filtered_df = df[(df["worked_with_python"] == 1) & (df["ConvertedCompYearly"].isna() == False)]
print(filtered_df.shape[0])

In [ ]:
# варіант 2
filtered_df = df[df.worked_with_python == 1].dropna(subset=["ConvertedCompYearly"])
print(filtered_df.shape[0])

In [ ]:
# варіант 3

# Фільтрація респондентів, які працюють з Python (Метод: query() використовується для фільтрації рядків у DataFrame на основі заданої умови. Умова вказується як рядок.)
python_devs = df.query("worked_with_python == 1").dropna(subset=["ConvertedCompYearly"])

# Групування за країною і розрахунок середнього та медіани
python_devs.groupby("Country")["ConvertedCompYearly"].agg(["mean", "median"])

In [ ]:
df[df.worked_with_python == 1].dropna(subset="ConvertedCompYearly").groupby(
    "Country",
    as_index=False,  # as_index=False - не робити індексом стовпець, по якому групуємо (не принципово!!!!)
).agg({"ConvertedCompYearly": ["mean", "median"]}).droplevel(
    1, axis=1
)  # droplevel(1, axis=1) - видаляємо перший рівень індексу (в даному випадку - "ConvertedCompYearly")

In [ ]:
# ще один варіант - додаємо ренейми стовпців і переводимо в int
py_progers_by_country = df[df.worked_with_python == 1].dropna(subset='ConvertedCompYearly') \
                                                      .groupby('Country', as_index=False) \
                                                      .agg({'ConvertedCompYearly': ['mean', 'median']}).droplevel(1, axis=1)
py_progers_by_country.columns = ['country', 'mean_comp', 'median_comp']
py_progers_by_country = py_progers_by_country.astype({'mean_comp': 'int', 'median_comp': 'int'})
py_progers_by_country

### Які рівні освіти мають 5 респондентів з найбільшою компенсацією?

In [ ]:
df[["ResponseId", "EdLevel", "ConvertedCompYearly"]].sort_values( by="ConvertedCompYearly", ascending=False).head(5)

In [ ]:
df[['ResponseId', 'EdLevel', 'ConvertedCompYearly']].sort_values(by='ConvertedCompYearly', ascending=False).reset_index(drop=True).head(5)

### В кожній віковій категорії, який відсоток респондентів програмує на Python?

In [ ]:
df.groupby("Age").agg({"ResponseId": "count", "worked_with_python": "sum"})

In [ ]:
df.groupby("Age", as_index=False).agg({"ResponseId": "count", "worked_with_python": "sum"})

In [ ]:
py_progers_share_by_age = df.groupby("Age", as_index=False).agg(
    {"ResponseId": "count", "worked_with_python": "sum"}
)  # as_index=False - не робити індексом стовпець, по якому групуємо (не принципово!!!!)
py_progers_share_by_age["work_with_python_share"] = round(
    py_progers_share_by_age.worked_with_python / py_progers_share_by_age.ResponseId, 2
)
py_progers_share_by_age

### Серед респондентів що знаходяться в 75 перцентилі за компенсацією середнього і працюють віддалено, які індустрії є найрозповсюдженішими?

In [ ]:
df[(df.ConvertedCompYearly > df.ConvertedCompYearly.quantile(0.75)) & (df.RemoteWork == 'Remote')].Industry.value_counts().reset_index()

# Чому краще звертатись до стовпців через квадратні дужки?

In [ ]:

df.head(2)
df['Oклад за рік, $']  = df['ConvertedCompYearly']

In [ ]:
df["Oклад за рік, $"]

In [ ]:
#df.Oклад за рік, $ # не працює, бо в назві стовпця є пробіл 